Load and clean data

In [ ]:
import sys
import time
from pathlib import Path
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV, LeaveOneGroupOut

# Set ROOT path to access other directories in project
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import SnowDepth.data_loader as DL
import SnowDepth.data_splitter as DS
import SnowDepth.optimal_features as OF
from SnowDepth.config import HOLDOUT_AOI
from SnowDepth.config import SEED
from SnowDepth.config import FEATURE_NAMES

In [ ]:
# Path to TIFF files
data_dir = ROOT/"data"/"tif_files"

# Select holdout AOI
holdout_aoi = HOLDOUT_AOI

# Select max amount of features to select from FF algos
top_k = 10

# Load dataframe
df = DL.build_df(str(data_dir), drop_invalid=True, upper_threshold=3)

# Development dataframe we will use for training models
dev_df  = df[df['aoi_name'] != holdout_aoi].copy()

In [ ]:
# Run Feature filtering algorithms
ff_algos = OF.optimal_feature_sets(dev_df, top_k=10, n_per_aoi=10000)
base_cols = ["aoi_name", "row", "col", "SD"]

# HSIC
dev_df_HSIC  = dev_df[base_cols + ff_algos["HSIC"]].copy()
# PCC
dev_df_PCC  = dev_df[base_cols + ff_algos["PCC"]].copy()
# MI
dev_df_MI  = dev_df[base_cols + ff_algos["MI"]].copy()
# ALL features (for Ablation study)
dev_df_ALL  = dev_df[base_cols + FEATURE_NAMES].copy()

Block HSIC Lasso B = 20.
M set to 3.
Using Gaussian kernel for the features, Gaussian kernel for the outcomes.
HSIC selected: ['Veg_height', 'IAFE', 'cos_Aspect', 'Gamma_VH_RTC', 'Gamma_VV_RTC', 'Sigma_ratio', 'Beta_VH', 'Gamma_RTC_ratio', 'Gamma_ratio', 'Slope']
PCC selected: ['Veg_height', 'IAFE', 'cos_Aspect', 'Gamma_VH_RTC', 'sin_Aspect', 'Gamma_RTC_ratio', 'Beta_VH']
MI selected): ['IAFE', 'Elevation', 'Veg_height', 'cos_Aspect', 'Gamma_RTC_sum', 'Gamma_VH_RTC', 'Gamma_VV_RTC', 'Slope', 'Gamma_ratio']


Split data for training XGBoost

In [ ]:
# HSIC
X_dev_HSIC, y_dev_HSIC, groups_HSIC = DS.ML_split(
    dev_df=dev_df_HSIC,
    pxs_per_aoi=10000
)
# PCC
X_dev_PCC, y_dev_PCC, groups_PCC = DS.ML_split(
    dev_df=dev_df_PCC,
    pxs_per_aoi=10000
)
# MI
X_dev_MI, y_dev_MI, groups_MI = DS.ML_split(
    dev_df=dev_df_MI,
    pxs_per_aoi=10000
)
# ALL
X_dev_ALL, y_dev_ALL, groups_ALL = DS.ML_split(
    dev_df=dev_df_ALL,
    pxs_per_aoi=10000
)

Total samples: 50000 across 5 AOIs
Features used: ['Veg_height', 'IAFE', 'cos_Aspect', 'Gamma_VH_RTC', 'Gamma_VV_RTC', 'Sigma_ratio', 'Beta_VH', 'Gamma_RTC_ratio', 'Gamma_ratio', 'Slope']
X_dev shape: (50000, 10)
Total samples: 50000 across 5 AOIs
Features used: ['Veg_height', 'IAFE', 'cos_Aspect', 'Gamma_VH_RTC', 'sin_Aspect', 'Gamma_RTC_ratio', 'Beta_VH']
X_dev shape: (50000, 7)
Total samples: 50000 across 5 AOIs
Features used: ['IAFE', 'Elevation', 'Veg_height', 'cos_Aspect', 'Gamma_RTC_sum', 'Gamma_VH_RTC', 'Gamma_VV_RTC', 'Slope', 'Gamma_ratio']
X_dev shape: (50000, 9)


Train XGBoost and tune hyperparameters

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, LeaveOneGroupOut, cross_validate
from xgboost import XGBRegressor
import numpy as np, time

param_dist = {
    "n_estimators":      [400, 600, 800, 1000],
    "learning_rate":     [0.03, 0.05, 0.07, 0.1, 0.15],
    "max_depth":         [4, 5, 6, 8, 10],
    "min_child_weight":  [1, 2, 4, 6, 8, 12, 16],
    "subsample":         [0.6, 0.7, 0.8, 0.9, 1.0],
    "colsample_bytree":  [0.6, 0.7, 0.8, 0.9, 1.0],
    "reg_lambda":        [0, 0.5, 1, 2, 5, 10],
    "reg_alpha":         [0, 1e-4, 1e-3, 1e-2, 0.1, 1],
    "max_bin":           [256, 512],
}

def run_xgb_search(tag, X, y, groups, n_iter=30, seed=SEED):
    xgb = XGBRegressor(
        objective="reg:squarederror",
        tree_method="hist",
        n_jobs=-1,
        random_state=seed,
        eval_metric="rmse", 
    )

    logo = LeaveOneGroupOut()
    search = RandomizedSearchCV(
        estimator=xgb,
        param_distributions=param_dist,
        n_iter=n_iter,
        cv=logo,
        scoring="neg_root_mean_squared_error", 
        n_jobs=-1,
        verbose=2,
        random_state=seed,
        refit=True
    )

    t0 = time.time()
    search.fit(X, y, groups=groups)
    elapsed = (time.time() - t0) / 60.0

    best_rmse = -search.best_score_
    best_params = search.best_params_

    # Re-evaluate with cross_validate to report both RMSE and MAE
    best_xgb = XGBRegressor(**best_params, objective="reg:squarederror",
                            tree_method="hist", n_jobs=-1, random_state=seed, eval_metric="rmse")
    scores = cross_validate(
        best_xgb, X, y, groups=groups, cv=logo, n_jobs=-1, verbose=0,
        scoring={"rmse": "neg_root_mean_squared_error", "mae": "neg_mean_absolute_error"}
    )
    cv_rmse = -np.mean(scores["test_rmse"])
    cv_mae  = -np.mean(scores["test_mae"])

    print(f"\nResults - XGBoost with {tag} feature set")
    print(f"Best hyperparameters: {best_params}")
    print(f"Best CV RMSE (search score): {best_rmse:.4f}")
    print(f"Cross-validated (re-fit) RMSE: {cv_rmse:.4f} | MAE: {cv_mae:.4f}")
    print(f"Training time: {elapsed:.2f} minutes")

    return tag, cv_rmse, cv_mae, best_params, elapsed


Fitting 5 folds for each of 30 candidates, totalling 150 fits

Results - XGBoost with HSIC feature set
Best hyperparameters: {'subsample': 1.0, 'reg_lambda': 0, 'reg_alpha': 0.01, 'n_estimators': 600, 'min_child_weight': 4, 'max_depth': 4, 'max_bin': 512, 'learning_rate': 0.03, 'colsample_bytree': 0.9}
Best CV RMSE: 0.4798
Training time: 2.99 minutes
Fitting 5 folds for each of 30 candidates, totalling 150 fits

Results - XGBoost with PCC feature set
Best hyperparameters: {'subsample': 0.8, 'reg_lambda': 1, 'reg_alpha': 0.01, 'n_estimators': 800, 'min_child_weight': 1, 'max_depth': 4, 'max_bin': 256, 'learning_rate': 0.07, 'colsample_bytree': 0.9}
Best CV RMSE: 0.4874
Training time: 2.15 minutes
Fitting 5 folds for each of 30 candidates, totalling 150 fits

Results - XGBoost with MI feature set
Best hyperparameters: {'subsample': 0.6, 'reg_lambda': 2, 'reg_alpha': 0.001, 'n_estimators': 1000, 'min_child_weight': 8, 'max_depth': 10, 'max_bin': 512, 'learning_rate': 0.05, 'colsample_bytr

In [ ]:
xgb_results = {} 
xgb_timings = {}

runs = [
    ("HSIC", X_dev_HSIC, y_dev_HSIC, groups_HSIC),
    ("PCC",  X_dev_PCC,  y_dev_PCC,  groups_PCC),
    ("MI",   X_dev_MI,   y_dev_MI,   groups_MI),
    ("ALL",  X_dev_ALL,  y_dev_ALL,  groups_ALL),  
]

for tag, X, y, g in runs:
    tag, rmse, mae, params, tmin = run_xgb_search(tag, X, y, g)
    xgb_results[tag] = (rmse, mae, params)
    xgb_timings[tag] = tmin

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Results - XGBoost with PCC feature set
PCC — Best hyperparameters: {'subsample': 1.0, 'reg_lambda': 10, 'reg_alpha': 1, 'n_estimators': 1000, 'min_child_weight': 2, 'max_depth': 6, 'max_bin': 256, 'learning_rate': 0.1, 'colsample_bytree': 0.6}
PCC — Best CV RMSE: 0.5190581262111664


print("\nCross-validation results (XGBoost):")
for name, (rmse, mae, params) in xgb_results.items():
    print(f"\n{name} — CV RMSE: {rmse:.4f} | CV MAE: {mae:.4f}")
    print(f"{name} — Best hyperparameters: {params}")
    print(f"{name} — Training time: {xgb_timings[name]:.2f} minutes")

best_xgb_method = min(xgb_results, key=lambda k: xgb_results[k][0])
best_xgb_rmse, best_xgb_mae, best_xgb_params = xgb_results[best_xgb_method]

print(f"\n🏆 Best feature set with XGBoost: {best_xgb_method} "
      f"(CV RMSE = {best_xgb_rmse:.4f}, CV MAE = {best_xgb_mae:.4f}, "
      f"time = {xgb_timings[best_xgb_method]:.2f} min)")
print(f"Best XGB hyperparameters: {best_xgb_params}")
